# 🧪 CUDA + Torch 1.8.1 + MMCV + MMDetection Setup (Miniconda, no Docker)
This notebook recreates your Dockerfile environment **using Miniconda** inside Jupyter. Cells are idempotent so you can rerun them safely.

## 0) (Optional) System build tools

In [ ]:
%%bash
set -euo pipefail

# Create conda activation helper
HELPER_SCRIPT=~/activate_msda.sh
cat > "$HELPER_SCRIPT" << 'EOF'
#!/bin/bash
set -e
# Disable nounset temporarily to avoid unbound ADDR2LINE inside activate hooks
set +u
export ADDR2LINE="$(command -v addr2line || true)"
export NM="$(command -v nm || true)"
export STRINGS="$(command -v strings || true)"
source ~/miniconda3/etc/profile.d/conda.sh
conda activate msda 2>/dev/null || true
set -u
EOF

chmod +x "$HELPER_SCRIPT"
echo "✅ Helper script created at $HELPER_SCRIPT"


In [ ]:
%%bash
set -Eeuo pipefail

# Helpful error message with line number and command
trap 'echo "❌ Error on line $LINENO: $BASH_COMMAND" >&2' ERR

# Only proceed if apt-get exists (likely running as root in a Debian/Ubuntu env)
if command -v apt-get >/dev/null 2>&1; then
  export DEBIAN_FRONTEND=noninteractive

  # Refresh package lists only if needed
  apt-get update -y

  # Minimal, reproducible toolchain. No "|| true" which hides errors.
  apt-get install -y --no-install-recommends \
    build-essential gcc-10 g++-10 pkg-config cmake make \
    ffmpeg curl ca-certificates git wget

  # Prefer explicit compilers without changing system-wide alternatives.
  export CC=/usr/bin/gcc-10
  export CXX=/usr/bin/g++-10
  "$CC" --version

  # ---- CUDA Toolkit (11.3 local installer) -----------------------------------
  # NOTE: CUDA 11.3 is old and may not fully support newer Ubuntu releases.
  # If you're on Ubuntu 24.04, consider using a newer CUDA version instead.
  CUDA_VERSION_DIR=/usr/local/cuda-11.3
  CUDA_RUNFILE=cuda_11.3.0_465.19.01_linux.run
  CUDA_URL=https://developer.download.nvidia.com/compute/cuda/11.3.0/local_installers/${CUDA_RUNFILE}

  if [ ! -d "$CUDA_VERSION_DIR" ]; then
    echo "⬇️  Downloading CUDA 11.3 toolkit runfile…"
    # Download only if missing, retry a few times
    if [ ! -f "$CUDA_RUNFILE" ]; then
      wget --progress=dot:giga -t 3 -O "$CUDA_RUNFILE" "$CUDA_URL"
      echo "✅ Download complete"
    fi
    chmod +x "$CUDA_RUNFILE"

    echo "⚙️  Installing CUDA 11.3 toolkit (silent)…"
    ./"$CUDA_RUNFILE" --silent --toolkit --override

    # Make libraries discoverable for dynamic linker
    if [ -d "$CUDA_VERSION_DIR/lib64" ]; then
      echo "$CUDA_VERSION_DIR/lib64" > /etc/ld.so.conf.d/cuda-11-3.conf
      ldconfig
    fi
  else
    echo "ℹ️  CUDA already present at $CUDA_VERSION_DIR — skipping install."
  fi

  # ---- Environment (no .bashrc sourcing; make it system-wide and idempotent) -
  # Add a small profile script that will be active for future shells/logins.
  cat >/etc/profile.d/cuda.sh <<'EOF'
# CUDA environment (installed by setup script)
if [ -d /usr/local/cuda-11.3 ]; then
  export CUDA_HOME=/usr/local/cuda-11.3
  # Prepend CUDA bin; keep /usr/bin and existing PATH
  case ":${PATH}:" in
    *":$CUDA_HOME/bin:"*) : ;;
    *) export PATH="$CUDA_HOME/bin:/usr/bin:${PATH:-}";;
  esac
  # LD_LIBRARY_PATH guard against unset var under set -u
  if [ -d "$CUDA_HOME/lib64" ]; then
    if [ -z "${LD_LIBRARY_PATH-}" ]; then
      export LD_LIBRARY_PATH="$CUDA_HOME/lib64"
    else
      case ":${LD_LIBRARY_PATH}:" in
        *":$CUDA_HOME/lib64:"*) : ;;
        *) export LD_LIBRARY_PATH="$CUDA_HOME/lib64:${LD_LIBRARY_PATH}";;
      esac
    fi
  fi
fi
EOF

  # Also export for the current non-login shell so you can use CUDA right away
  export CUDA_HOME="$CUDA_VERSION_DIR"
  export PATH="$CUDA_HOME/bin:/usr/bin:${PATH:-}"
  if [ -d "$CUDA_HOME/lib64" ]; then
    export LD_LIBRARY_PATH="$CUDA_HOME/lib64:${LD_LIBRARY_PATH-}"
  fi

  # Show final toolchain info
  echo "🔧 Using CC=$CC  CXX=$CXX"
  which gcc || true
  gcc --version || true
  nvcc --version || { echo "⚠️  nvcc not found (CUDA bin not in PATH or install failed)"; exit 1; }

else
  echo "Skipping apt-get (not available). Ensure a C/C++ compiler is installed."
fi


## 1) Install Miniconda (user-local, idempotent)

In [ ]:
%%bash
set -euo pipefail
CONDA_HOME="$HOME/miniconda3"
if [ ! -d "$CONDA_HOME" ]; then
  echo "Installing Miniconda to $CONDA_HOME ..."
  wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
  bash /tmp/miniconda.sh -b -p "$CONDA_HOME"
  rm -f /tmp/miniconda.sh
else
  echo "Miniconda already present at $CONDA_HOME"
fi
# Initialize conda for bash in a user-safe way
"$CONDA_HOME/bin/conda" init bash >/dev/null 2>&1 || true
echo "Miniconda ready."

## 2) Configure conda solver (libmamba) for speed

In [ ]:
%%bash
set -euo pipefail
CONDA_HOME="$HOME/miniconda3"
eval "$($CONDA_HOME/bin/conda shell.bash hook)"

# Ensure the 'conda tos' command exists (older installers may lack it)
conda update -n base -y conda || true

# Accept ToS for Anaconda default channels
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main || true
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r || true

echo "Accepted Anaconda ToS for required channels."

conda install -y -n base conda-libmamba-solver || true
conda config --set solver libmamba || true
conda --version; conda config --show solver | sed -n 's/^solver: /solver: /p'

## 3) Create env `msda` (Python 3.9)

In [ ]:
%%bash
set -euo pipefail
CONDA_HOME="$HOME/miniconda3"
eval "$($CONDA_HOME/bin/conda shell.bash hook)"
if conda env list | awk '{print $1}' | grep -qx msda; then
  echo "Environment 'msda' already exists."
else
  conda create -y -n msda python=3.9
fi

## 4) Register Jupyter kernel for the env

In [ ]:
%%bash
set -euo pipefail
CONDA_HOME="$HOME/miniconda3"
eval "$($CONDA_HOME/bin/conda shell.bash hook)"
conda activate msda
python -m pip install --upgrade pip
python -m pip install ipykernel
python -m ipykernel install --user --name msda --display-name "Python (msda)" || true
echo "Kernel 'Python (msda)' installed/updated."

## 5) Install gcc 10

In [ ]:
%%bash
set -Eeuo pipefail
trap 'echo "❌ line $LINENO: $BASH_COMMAND" >&2' ERR

MINICONDA="$HOME/miniconda3"
source "$MINICONDA/etc/profile.d/conda.sh"  # only for the 'conda' command itself

conda create -y -n msda python=3.9 >/dev/null 2>&1 || true
conda install -y -n msda -c conda-forge "gcc_linux-64=10.*" "gxx_linux-64=10.*" binutils_linux-64

# Run your build commands inside the env without activating it in this shell
conda run -n msda bash -lc '
  set -euo pipefail
  export CC="$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-gcc"
  export CXX="$CONDA_PREFIX/bin/x86_64-conda-linux-gnu-g++"
  export CUDAHOSTCXX="$CXX"
  export PATH="$CONDA_PREFIX/bin:$PATH"
  "$CC" --version
  "$CXX" --version
'


## 5) Core Python stack (Torch 1.8.1 + cu111, MMCV, MMDetection)

In [ ]:
%%bash
set -Ee -o pipefail
trap 'echo "❌ line $LINENO: $BASH_COMMAND" >&2' ERR

MINICONDA="$HOME/miniconda3"
source "$MINICONDA/etc/profile.d/conda.sh"

set +u
conda activate msda
set -u

echo "numpy<2" > /tmp/constraints.txt

echo "📦 Installing PyTorch 1.8.1 + cu111..."
python -m pip install --no-cache-dir \
  --extra-index-url https://download.pytorch.org/whl/cu111 \
  torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1

echo "📦 Installing core libraries..."
python -m pip install --no-cache-dir -c /tmp/constraints.txt \
  "numpy==1.23.5" "opencv-python-headless<5" yacs pyyaml tqdm matplotlib pillow cython

#python -m pip install --no-cache-dir "setuptools<81"

echo "📦 Installing pycocotools separately..."
python -m pip install --no-cache-dir pycocotools

echo "📦 Installing build helpers..."
python -m pip install --no-cache-dir -c /tmp/constraints.txt \
  "pybind11>=2.12" "packaging<24" "scipy<1.11"

echo "📦 Installing mmcv-full with GCC workaround..."
# Set environment variables to allow unsupported compiler
export MMCV_WITH_OPS=1
export FORCE_CUDA=1
# Use older GCC if available, otherwise allow unsupported
if command -v gcc-10 &> /dev/null; then
    export CC=gcc-10
    export CXX=g++-10
    echo "Using GCC 10"
else
    export TORCH_NVCC_FLAGS="-allow-unsupported-compiler"
    echo "Using -allow-unsupported-compiler flag"
fi

python -m pip install --no-cache-dir -c /tmp/constraints.txt \
  --no-build-isolation \
  "mmcv-full==1.7.2" \
  -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.1/index.html

echo "📦 Installing mmdet..."
python -m pip install --no-cache-dir -c /tmp/constraints.txt "mmdet==2.28.2"

echo "✅ Installation complete!"
python -c "import torch; import mmcv; import mmdet; print(f'PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}, mmcv: {mmcv.__version__}, mmdet: {mmdet.__version__}')"

## 6) (Optional) MEGAcmd

In [ ]:
%%bash
set -Eeuo pipefail
trap 'echo "❌ line $LINENO: $BASH_COMMAND" >&2' ERR

# Choose a single canonical keyring filename
KEYRING=/usr/share/keyrings/mega-archive-keyring.gpg

# Detect Ubuntu version to pick the right MEGA subpath
. /etc/os-release
case "${VERSION_ID}" in
  20.04) REPO_SUBDIR="xUbuntu_20.04" ;;  # focal
  22.04) REPO_SUBDIR="xUbuntu_22.04" ;;  # jammy
  24.04) REPO_SUBDIR="xUbuntu_24.04" ;;  # noble
  *)     REPO_SUBDIR="xUbuntu_20.04" ;;  # fallback to your original
esac

echo "Using repo path: https://mega.nz/linux/repo/${REPO_SUBDIR}/ ./"
echo "Using keyring:   $KEYRING"

# Ensure tools
sudo apt-get update -y || true
sudo apt-get install -y --no-install-recommends curl gnupg ca-certificates

# 1) Remove ALL existing MEGA lines from any apt source file
for f in /etc/apt/sources.list /etc/apt/sources.list.d/*.list; do
  [ -f "$f" ] || continue
  if grep -q 'mega\.nz/linux/repo' "$f"; then
    echo "Cleaning MEGA entries from $f"
    sudo cp -a "$f" "$f.bak.$(date +%s)"
    sudo sed -i '/mega\.nz\/linux\/repo/d' "$f"
  fi
done

# 2) Remove both old/new keyrings so we can recreate a single consistent one
sudo rm -f /usr/share/keyrings/meganz-archive-keyring.gpg
sudo rm -f /usr/share/keyrings/mega-archive-keyring.gpg

# 3) Re-import MEGA key into our canonical keyring
curl -fsSL https://mega.nz/keys/MEGA_signing.key \
  | gpg --dearmor \
  | sudo tee "$KEYRING" >/dev/null

# 4) Create a single repo file with our chosen keyring
echo "deb [signed-by=$KEYRING] https://mega.nz/linux/repo/${REPO_SUBDIR}/ ./" \
  | sudo tee /etc/apt/sources.list.d/megacmd.list >/dev/null

# 5) Update and install
sudo apt-get update -y
sudo apt-get install -y --no-install-recommends megacmd

echo "✅ MEGAcmd installed cleanly."


## 7) (Optional) Set CUDA vars for builds

In [ ]:
%%bash
set -euo pipefail
CONDA_HOME="$HOME/miniconda3"
eval "$($CONDA_HOME/bin/conda shell.bash hook)"
set +u
conda activate msda
set -u
if [ -d /usr/local/cuda ]; then
  mkdir -p "$CONDA_PREFIX/etc/conda/activate.d"
  echo 'export CUDA_HOME=/usr/local/cuda' >> "$CONDA_PREFIX/etc/conda/activate.d/env_vars.sh"
  echo 'export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH' >> "$CONDA_PREFIX/etc/conda/activate.d/env_vars.sh"
  echo "CUDA vars will be set when 'msda' is activated."
else
  echo "No /usr/local/cuda detected."
fi

## 8) Sanity check (run with kernel: Python (msda))

In [ ]:
%%bash
set -euo pipefail
CONDA_HOME="$HOME/miniconda3"
eval "$($CONDA_HOME/bin/conda shell.bash hook)"
set +u
conda activate msda
set -u

python -c "
import os, torch, numpy, cv2, mmcv, mmdet
print('torch:', torch.__version__, 'cuda:', torch.version.cuda, 'avail:', torch.cuda.is_available())
print('numpy:', numpy.__version__)
print('opencv:', cv2.__version__)
print('mmcv:', mmcv.__version__, 'mmdet:', mmdet.__version__)
print('CUDA_HOME:', os.environ.get('CUDA_HOME'))
"

## 9) (Optional) Build your project ops

In [ ]:
%%bash
set -euo pipefail

CONDA_HOME="$HOME/miniconda3"

# --- prevent unbound vars inside conda's activate.d hooks ---
set +u
export ADDR2LINE="${ADDR2LINE-$(command -v addr2line || true)}"
export NM="${NM-$(command -v nm || true)}"
export STRINGS="${STRINGS-$(command -v strings || true)}"
eval "$($CONDA_HOME/bin/conda shell.bash hook)"
conda activate msda
set -u
# ------------------------------------------------------------

# echo "Setting up repositories..."

# REPO_DIR="$HOME/TemporalAttentionPlayground"
# if [ ! -d "$REPO_DIR" ]; then
#     echo "Cloning TemporalAttentionPlayground..."
#     git clone --recurse-submodules https://github.com/mozi30/TemporalAttentionPlayground.git "$REPO_DIR"
# else
#     echo "TemporalAttentionPlayground already exists, skipping clone."
# fi

export CC=gcc-10
export CXX=g++-10
echo "Using GCC 10"
which gcc
which g++

OPS_DIR="$HOME/TemporalAttentionPlayground/TransVOD_plusplus/models/ops"
if [ -d "$OPS_DIR" ]; then
    echo "Building TransVOD_plusplus ops..."
    cd "$OPS_DIR"
    python setup.py install
else
    echo "Warning: $OPS_DIR does not exist. Skipping ops build."
fi

echo "✅ Setup complete!"


## 10) Download Visdrone dataset

In [ ]:
%%bash
source ~/activate_msda.sh
REPO_DIR="$HOME/TemporalAttentionPlayground"
export MEGA_EMAIL="e12217036@student.tuwien.ac.at"
export MEGA_PASS="Spam_pass30"
python3 $REPO_DIR/downloadVisdrone.py --root $HOME/datasets-visdrone


In [ ]:
## 11) Convert Visdrone dataset to Transvod input format

In [ ]:
%%bash
source ~/activate_msda.sh
REPO_DIR="$HOME/TemporalAttentionPlayground"
echo $HOME
#Fix spelling mistake of Visdrone and give better name overall
python3 $REPO_DIR/testVisdoneToImageNetVid.py \
--visdrone-train $HOME/datasets-visdrone/VisDrone2019-VID-train \
--visdrone-val $HOME/datasets-visdrone/VisDrone2019-VID-val \
--out-root $HOME/datasets/visdrone/transvod \
--link-mode hardlink


In [ ]:
%%bash
source ~/activate_msda.sh
REPO_DIR="$HOME/TemporalAttentionPlayground"
conda list

python3 $REPO_DIR/TransVOD_plusplus/main.py --dataset_file vid_single --vid_path $HOME/datasets/visdrone/transvod

## Build YOLOX

In [ ]:
%%bash
set -euo pipefail
CONDA_HOME="$HOME/miniconda3"
eval "$($CONDA_HOME/bin/conda shell.bash hook)"
if conda env list | awk '{print $1}' | grep -qx msda; then
  echo "Environment 'yolox' already exists."
else
  conda create -y -n yolox python=3.9
fi

In [ ]:
%%bash
set -euo pipefail

# Create conda activation helper
HELPER_SCRIPT=~/activate_yolox.sh
cat > "$HELPER_SCRIPT" << 'EOF'
#!/bin/bash
set -e
# Disable nounset temporarily to avoid unbound variables in conda hooks
set +u
export ADDR2LINE="$(command -v addr2line || true)"
export NM="$(command -v nm || true)"
export STRINGS="$(command -v strings || true)"
# Pre-initialize libxml2 variable to prevent deactivate.d script errors
export xml_catalog_files_libxml2="${xml_catalog_files_libxml2:-}"
source ~/miniconda3/etc/profile.d/conda.sh
conda activate yolox 2>/dev/null || true
set -u
EOF

chmod +x "$HELPER_SCRIPT"
echo "✅ Helper script created at $HELPER_SCRIPT"

In [ ]:
%%bash
# Disable strict mode temporarily for conda activation
set -eo pipefail
source ~/activate_yolox.sh

# Re-enable strict mode after activation
set -u

# ==========================
# 📦 YOLOX INSTALLATION - TESTED COMPATIBLE VERSIONS
# ==========================
# Based on YOLOX v0.3.0 release (April 2022 timeframe)
# Python 3.7-3.9 recommended

echo "============================"
echo "Step 1: Installing PyTorch 1.10.0 + CUDA 11.3"
echo "============================"
# PyTorch 1.10.0 is the sweet spot - stable and widely compatible
pip install --no-cache-dir \
    torch==1.10.0+cu113 \
    torchvision==0.11.1+cu113 \
    --extra-index-url https://download.pytorch.org/whl/cu113

echo "============================"
echo "Step 2: Installing core dependencies from conda"
echo "============================"
# Install from conda for better binary compatibility
# Install numpy first with version constraint to prevent scipy issues
conda install -y -c conda-forge \
    "numpy=1.21.5" \
    opencv=4.5.5 \
    scikit-image=0.19.1 \
    pillow=9.0.1 \
    h5py=3.6.0 \
    scikit-learn=1.0.2 \
    "scipy=1.7.3" \
    cython

echo "============================"
echo "Step 3: Installing pip dependencies"
echo "============================"
pip install --no-cache-dir \
    loguru==0.6.0 \
    tqdm==4.62.3 \
    ninja==1.10.2 \
    tabulate==0.8.9 \
    tensorboard==2.8.0 \
    thop==0.0.31-2005241907

echo "============================"
echo "Step 4: Installing tracking libraries"
echo "============================"
# Try conda's prebuilt lap with numpy pinned to prevent upgrades
echo "Attempting conda install of lap with numpy pinned..."
conda install -y -c conda-forge lap "numpy=1.21.5" && LAP_INSTALLED=true || LAP_INSTALLED=false

# If conda succeeded, verify numpy version is still correct
if [ "$LAP_INSTALLED" = "true" ]; then
    NUMPY_VER=$(python -c "import numpy; print(numpy.__version__)")
    if [[ ! "$NUMPY_VER" =~ ^1\.21\. ]]; then
        echo "⚠️  Conda lap changed numpy to $NUMPY_VER, removing and trying pip..."
        conda remove -y lap
        LAP_INSTALLED=false
    fi
fi

# If conda failed or changed numpy, try pip with constraints
if [ "$LAP_INSTALLED" = "false" ]; then
    echo "Trying pip install with constraints..."
    # Create constraints file to prevent numpy upgrade
    cat > /tmp/lap-constraints.txt << 'EOF'
numpy==1.21.5
scipy==1.7.3
EOF
    pip install --no-cache-dir -c /tmp/lap-constraints.txt "lapx>=0.5.4" || \
        echo "⚠️  All lap installation methods failed"
fi

# Verify numpy version hasn't changed
python -c "import numpy; assert numpy.__version__.startswith('1.21'), f'Wrong numpy: {numpy.__version__}'"

# Install other tracking libraries
pip install --no-cache-dir \
    motmetrics==1.2.5 \
    filterpy==1.4.5

echo "============================"
echo "Step 5: Installing COCO tools"
echo "============================"
# Try conda first for pycocotools (has pre-built binary)
conda install -y -c conda-forge pycocotools "numpy=1.21.5" || \
    pip install --no-cache-dir pycocotools==2.0.6 || \
    pip install --no-cache-dir pycocotools==2.0.4 || \
    echo "⚠️  pycocotools installation failed"

echo "============================"
echo "Step 6: Installing ONNX (skip if it fails - not critical for training)"
echo "============================"
# Try prebuilt wheel first, fall back to conda if fails
pip install --no-cache-dir onnxruntime==1.10.0 || \
    conda install -y -c conda-forge onnxruntime=1.10.0

# Try to install onnx, but don't fail if it doesn't work
pip install --no-cache-dir onnx==1.10.2 onnx-simplifier==0.4.1 || \
    echo "⚠️  ONNX installation failed (non-critical for training)"

echo "============================"
echo "✅ Installation complete!"
echo "============================"
python -c "
import torch, numpy, scipy
print(f'PyTorch: {torch.__version__}')
print(f'NumPy: {numpy.__version__}')
print(f'SciPy: {scipy.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
"
python -c "
try:
    import lap
    print('✅ lap module imported successfully')
except ImportError:
    try:
        import lapx
        print('✅ lapx module imported successfully (alternative to lap)')
    except ImportError:
        print('⚠️  No lap module available - may affect tracking performance')
"
python -c "
try:
    import pycocotools
    print('✅ pycocotools module imported successfully')
except ImportError:
    print('⚠️  pycocotools not available - needed for COCO evaluation')
"
echo ""
echo "Next step: cd /root/TemporalAttentionPlayground/YOLOV && pip install -v -e ."

In [ ]:
%%bash
set -eo pipefail
source ~/activate_yolox.sh
set -u

echo "============================"
echo "Setting up YOLOX package"
echo "============================"
cd /root/TemporalAttentionPlayground/YOLOV

echo "All dependencies are already installed from Step 6."
echo "Adding YOLOX to Python path instead of pip install to avoid dependency conflicts."
echo ""

# Create a .pth file to add YOLOX to Python path
SITE_PACKAGES=$(python -c "import site; print(site.getsitepackages()[0])")
echo "/root/TemporalAttentionPlayground/YOLOV" > "$SITE_PACKAGES/yolox.pth"

echo "✅ YOLOX path configured: $SITE_PACKAGES/yolox.pth"
echo ""

# Verify YOLOX can be imported
echo "Verifying installation..."
python -c "
import sys
import yolox
print(f'✅ YOLOX version: {yolox.__version__}')
print(f'✅ YOLOX path: {yolox.__file__}')
"

echo ""
python -c "
import torch
print(f'✅ PyTorch: {torch.__version__}')
print(f'✅ CUDA available: {torch.cuda.is_available()}')
"

echo ""
python -c "
try:
    import onnx
    print(f'✅ ONNX: {onnx.__version__}')
except ImportError:
    print('⚠️  ONNX not available (not required for training)')
"

echo ""
echo "============================"
echo "✅ YOLOX setup complete!"
echo "============================"
echo ""
echo "You can now use YOLOX for training without any ONNX 1.8.1 issues."
echo "For model export, the newer ONNX 1.10.2 should work fine."

In [ ]:
%%bash
set -eo pipefail
source ~/activate_yolox.sh
set -u

echo "============================"
echo "Installing missing YOLOX dependency: timm"
echo "============================"
# YOLOX's Swin Transformer backbone requires timm (PyTorch Image Models)
pip install --no-cache-dir timm==0.9.16

echo ""
echo "============================"
echo "Testing YOLOX train.py"
echo "============================"
cd /root/TemporalAttentionPlayground/YOLOV

# Test that train.py can be imported and shows help
python tools/train.py --help | head -30

echo ""
echo "============================"
echo "✅ YOLOX is ready for training!"
echo "============================"

In [ ]:
#Download visdrone from my ftp server
from ftplib import FTP, error_perm
from pathlib import Path
#import getpass

FTP_HOST = "minecraftwgwg.hopto.org"
FTP_USER = "ftpuser"
FTP_PASS = "ftpadmin"  # For security, consider using getpass.getpass()
REMOTE_PATH = "datasets/visdrone"
LOCAL_PATH = str(Path.home() / "TemporalAttentionPlayground" / "datasets" / "visdrone")
TEST_SET_NAME = "VisDrone2019-VID-test"
VAL_SET_NAME = "VisDrone2019-VID-val"
TRAIN_SET_NAME = "VisDrone2019-VID-train"


def download_file_ftp(host, username, password, remote_path, local_path):
    local_path = Path(local_path)
    local_path.parent.mkdir(parents=True, exist_ok=True)

    with FTP(host, timeout=30) as ftp:
        ftp.login(user=username, passwd=password)         # authenticate
        print(f"Connected to FTP: {host} as {username}")
        ftp.set_pasv(True)                                # passive mode (default)
        print(f"Changed to passive mode.")
        # print current dir and list
        print("Server welcome:", ftp.getwelcome())
        print("Listing:", ftp.nlst())                     # list directory (filenames)

        # download binary file
        with open(local_path, "wb") as f:
            def callback(chunk):
                f.write(chunk)
            ftp.retrbinary(f"RETR {remote_path}", callback)
        print(f"Downloaded: {remote_path} to {local_path}")


# Download train, val, and test sets
print("Starting dataset downloads...")
for dataset_name in [TRAIN_SET_NAME, VAL_SET_NAME, TEST_SET_NAME]:
    remote_file = f"{REMOTE_PATH}/{dataset_name}.zip"
    local_file = f"{LOCAL_PATH}/{dataset_name}.zip"
    try:
        download_file_ftp(FTP_HOST, FTP_USER, FTP_PASS, remote_file, local_file)
    except error_perm as e:
        print(f"FTP error downloading {remote_file}: {e}")
    except Exception as e:
        print(f"Error downloading {remote_file}: {e}")
print("All downloads attempted.")

In [ ]:
%%bash
source ~/activate_msda.sh
REPO_DIR="$HOME/TemporalAttentionPlayground"
echo $HOME
#Fix spelling mistake of Visdrone and give better name overall
python3 $REPO_DIR/testVisdroneToImageNetVid.py \
--visdrone-train $REPO_DIR/datasets/visdrone/VisDrone2019-VID-train \
--visdrone-val $REPO_DIR/datasets/visdrone/VisDrone2019-VID-val \
--out-root $REPO_DIR/datasets/visdrone/transvod \
--link-mode hardlink